In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import plotly.express as px

# 设置随机种子以确保可重复性
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# 公司名称列表
company_name_list = [
    "湖北国贸能源化工有限公司",
    "湖北国贸金属矿产有限公司",
    "湖北国贸汽车有限公司",
    "湖北国际贸易集团有限公司",
    "湖北国贸农产品有限公司"
]

# 生成随机客户名称
customer_names = [f"客户{i}" for i in range(1, 51)]

# 生成数据
n_rows = 1000
now = datetime.now()
data = {
    "公司名称": np.random.choice(company_name_list, n_rows),
    "客户名称": np.random.choice(customer_names, n_rows),
    "回款金额": np.random.uniform(100000, 5000000, n_rows).round(2),
    "回款日期": [
        (now - timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d")
        for _ in range(n_rows)
    ],
}

# 创建 DataFrame
df = pd.DataFrame(data)

# 创建公司名称简称的字典
company_short_names = {
    "湖北国贸能源化工有限公司": "国贸能化",
    "湖北国贸金属矿产有限公司": "国贸金属矿",
    "湖北国贸汽车有限公司": "国贸汽车",
    "湖北国际贸易集团有限公司": "国贸集团",
    "湖北国贸农产品有限公司": "国贸农产品"
}

# 将简称添加到DataFrame中
df['公司简称'] = df['公司名称'].map(company_short_names)

# 将回款日期列转换为datetime类型
df['回款日期'] = pd.to_datetime(df['回款日期'])

def set_mobile_layout(fig, title, x_title, y_title):
    fig.update_layout(
        autosize=False,
        width=360,
        height=300,
        margin=dict(l=10, r=10, t=60, b=40),
        title_font_size=16,
        font=dict(size=12),
        title=dict(text=title, x=0.3),
        xaxis=dict(
            title=x_title,
            tickangle=-45,
            title_font=dict(size=14),
            tickfont=dict(size=10)
        ),
        yaxis=dict(
            title=y_title,
            title_font=dict(size=14),
            tickfont=dict(size=10),
            tickformat='.2s'
        )
    )
    return fig

def calculate_bar_width(n_bars, chart_width=360, min_width=0.5, max_width=0.8):
    ideal_width = (chart_width * 0.8) / (n_bars + 1)
    return max(min(ideal_width / chart_width, max_width), min_width)

current_year = pd.Timestamp.now().year

# 例子1: 集团用户查询各公司的回款情况
df_current_year = df[df['回款日期'].dt.year == current_year]
df_company_payment = df_current_year.groupby('公司名称')['回款金额'].sum().sort_values(ascending=False).reset_index()
df_company_payment['公司简称'] = df_company_payment['公司名称'].map(company_short_names)

fig1 = px.bar(df_company_payment, x='公司简称', y='回款金额')
fig1 = set_mobile_layout(fig1, 
                         f'{current_year}年<br>各公司回款金额总和', 
                         '公司名称', '回款金额 (元)')
fig1.update_xaxes(type='category', categoryorder='total descending')
fig1.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_company_payment)))
fig1.show()

# 例子2: 公司用户查询客户回款情况
company = "湖北国贸能源化工有限公司"
df_company = df_current_year[df_current_year['公司名称'] == company]
df_customer_payment = df_company.groupby('客户名称')['回款金额'].sum().sort_values(ascending=False).reset_index()

fig2 = px.bar(df_customer_payment, x='客户名称', y='回款金额')
fig2 = set_mobile_layout(fig2, 
                         f'{company_short_names[company]} {current_year}年<br>各客户回款金额', 
                         '客户名称', '回款金额 (元)')
fig2.update_xaxes(type='category', categoryorder='total descending')
fig2.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_customer_payment)))
fig2.show()
# 例子3: 集团用户按条件查询特定公司的回款情况
company = "湖北国贸金属矿产有限公司"
df_company = df_current_year[df_current_year['公司名称'] == company]
df_customer_payment_filtered = df_company[df_company['回款金额'] > 4000000].groupby('客户名称')['回款金额'].sum().sort_values(ascending=False).reset_index()

fig3 = px.bar(df_customer_payment_filtered, x='客户名称', y='回款金额')
fig3 = set_mobile_layout(fig3, 
                         f'{company_short_names[company]} {current_year}年<br>单笔回款金额超过400万的客户', 
                         '客户名称', '回款金额 (元)')
fig3.update_xaxes(type='category', categoryorder='total descending')
fig3.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_customer_payment_filtered)))
fig3.show()